# Ex 10: The moment of inertia tensor of a Space Station

Let's assume a space station define by the given modules:

Module|Mass|x|y|z
--|--|--|--|--
Astrophysics module|3|6|-2|0
Robotics repair plant|4|3|1|-2
Power plant|7|-4|1|2
Docking module|5|-4|-2|-4
Communications module|4|3|2|-1
Solar arrays|3|2|-1|6

We can then calculate the total mass, center of mass and inertia matrix of the station.

In [1]:
import numpy as np
from numpy.linalg import eig

modules = np.array([[3,  6, -2,  0],
                    [4,  3,  1, -2],
                    [7, -4,  1,  2],
                    [5, -4, -2, -4],
                    [4,  3,  2, -1],
                    [3,  2, -1,  6]])

total_mass = np.sum(modules, axis=0)[0]
total_mass

26

The center of mass of the space station can be find using the mass and position off all the submodules:

$$x_{com}=\frac{\sum{m_i*x_i}}{m_{total}}$$
$$y_{com}=\frac{\sum{m_i*y_i}}{m_{total}}$$
$$z_{com}=\frac{\sum{m_i*z_i}}{m_{total}}$$

In [2]:
center_of_mass = (modules[:, 1:].transpose() * modules[:, 0]).sum(axis=1) / total_mass
center_of_mass

array([0., 0., 0.])

Center of mass is at the origin of the coordinate system.

We can find the inertia matrix of the system with:

$$
I=
\left(
\begin{array}{ccc}
I_{xx} & I_{xy} & I_{xz} \\
I_{xy} & I_{yy} & I_{yz} \\
I_{xz} & I_{yz} & I_{zz} \\
\end{array}
\right)
=
\left(
\begin{array}{ccc}
\sum{m(y^2 + z^2)} & -\sum{mxy}         & -\sum{mxz} \\
-\sum{mxy}         & \sum{m(x^2 + z^2)} & -\sum{myz} \\
-\sum{mxz}         & -\sum{myz}         & \sum{m(x^2 + y^2)} \\
\end{array}
\right)
$$

In [3]:
def inertia_moment(mass, x, y):
    return mass * (x**2 + y**2)
def inertia_product(mass, x, y):
    return mass * x * y

inertia = np.array([[0, 0, 0],
                    [0, 0, 0],
                    [0, 0, 0]])

for m, x, y, z in modules:
    inertia += np.array([[inertia_moment(m, y, z), -inertia_product(m, x, y), -inertia_product(m, x, z)],
                         [-inertia_product(m, x, y), inertia_moment(m, x, z), -inertia_product(m, y, z)],
                         [-inertia_product(m, x, z), -inertia_product(m, y, z), inertia_moment(m, x, y)]])

inertia

array([[298,  -6, -24],
       [ -6, 620, -20],
       [-24, -20, 446]])

All symmetrical inertia matrices can be diagonalised using eig vectors as the new base and the moments of inertia for these primary directions will be the corresponding eig values.

In [4]:
w,v=eig(inertia)
print('E-value:', w)
print('E-vector', v)

E-value: [293.95245163 447.74525937 622.302289  ]
E-vector [[-0.98681235 -0.16155017  0.01014485]
 [-0.02793964  0.10826326 -0.99372956]
 [-0.15943887  0.98090805  0.11134918]]


The eigenvalues and eigenvectors allow us to create a new set of axes, forming a basis, from which we can express the inertia matrix in diagonal form. Fortunately, since the inertia matrix was nearly diagonal to begin with, the eigenvectors are similar to the initial basis, making interpretation of the results easier.

Therefore, the space station exhibits the greatest inertia around the last eigenvector (primarily the y-axis) and the least inertia around the first eigenvector (primarily the x-axis). The station's stability is directly proportional to its inertia. Therefore, the greater the inertia, the more stable the station is when spinning around its axis

## New Solar arrays mass

Now, let's update the modules tables with the below values and find the new center of mass and inertia matrix.

Module|Mass|x|y|z
--|--|--|--|--
Astrophysics module|3|6|-2|0
Robotics repair plant|4|3|1|-2
Power plant|7|-4|1|2
Docking module|5|-4|-2|-4
Communications module|4|3|2|-1
Solar arrays|5|2|-1|6

In [5]:
modules = np.array([[3,  6, -2,  0],
                    [4,  3,  1, -2],
                    [7, -4,  1,  2],
                    [5, -4, -2, -4],
                    [4,  3,  2, -1],
                    [5,  2, -1,  6]])

total_mass = np.sum(modules, axis=0)[0]
total_mass

28

In [6]:
center_of_mass = (modules[:, 1:].transpose() * modules[:, 0]).sum(axis=1) / total_mass
center_of_mass

array([ 0.14285714, -0.07142857,  0.42857143])

In [7]:
inertia = np.array([[0, 0, 0],
                    [0, 0, 0],
                    [0, 0, 0]], dtype=float)

for m, x, y, z in modules:
    inertia += np.array([[inertia_moment(m, y-center_of_mass[1], z-center_of_mass[2]), -inertia_product(m, x-center_of_mass[0], y-center_of_mass[1]), -inertia_product(m, x-center_of_mass[0], z-center_of_mass[2])],
                        [-inertia_product(m, x-center_of_mass[0], y-center_of_mass[1]), inertia_moment(m, x-center_of_mass[0], z-center_of_mass[2]), -inertia_product(m, y-center_of_mass[1], z-center_of_mass[2])],
                        [-inertia_product(m, x-center_of_mass[0], z-center_of_mass[2]), -inertia_product(m, y-center_of_mass[1], z-center_of_mass[2]), inertia_moment(m, x-center_of_mass[0], y-center_of_mass[1])]])

inertia

array([[366.71428571,  -2.28571429, -46.28571429],
       [ -2.28571429, 694.28571429,  -8.85714286],
       [-46.28571429,  -8.85714286, 455.28571429]])

In [10]:
w_new,v_new=eig(inertia)
print('E-value:', w_new)
print('E-vector', v_new)

E-value: [346.85094272 474.82024129 694.61453027]
E-vector [[-0.91912237 -0.39396808  0.00179457]
 [-0.0160819   0.03296681 -0.99932705]
 [-0.3936438   0.91853271  0.03663629]]


In [9]:
w_new - w

array([52.89849109, 27.07498192, 72.31224128])

As anticipated, the moment of inertia increases with the mass of the solar arrays, making the station more resistant to external perturbations. However, even a small change in mass can have a significant impact on the inertia of the entire system.

Additionally, since the solar panels are located close to the center on the xy plane, changes in mass will have a lesser effect on the z-axis compared to the other axes. These findings are supported by the new eigenvalues and eigenvectors.